## Top 10 items by sales/units

In [1]:
import duckdb

con=duckdb.execute("""IMPORT DATABASE '../data/db';""")
df = con.execute(""" SELECT item_id, SUM(qty_day) units FROM daily_demand
  GROUP BY item_id
  ORDER BY units DESC
  LIMIT 10;""").df()
con.close()
df

,item_id,units
0,I002,12027.0
1,I011,11879.0
2,I012,11861.0
3,I006,7042.0
4,I014,6998.0
5,I007,6995.0
6,I008,6942.0
7,I013,6928.0
8,I004,4991.0
9,I003,4985.0


In [2]:
con=duckdb.execute("""IMPORT DATABASE '../data/db';""")
df = con.execute(""" SELECT
     doy,
     ship_id,
     SUM(qty_day) AS total_sales
 FROM daily_demand
 WHERE item_id = 'I002'
 GROUP BY doy, ship_id
 ORDER BY doy, ship_id;""").df()
con.close()
df

,doy,ship_id,total_sales
0,121,S1,28.0
1,121,S2,18.0
2,121,S3,8.0
3,122,S1,63.0
4,122,S2,43.0
...,...,...,...
249,204,S1,55.0
250,204,S2,25.0
251,204,S3,34.0
252,205,S1,27.0


In [17]:
import plotly.express as px

# Ensure data is sorted by day of year
_df_plot = df.sort_values(["doy", "ship_id"])

fig = px.line(
    _df_plot,
    x="doy",
    y="total_sales",
    color="ship_id",
    markers=True,
    title="Total Sales by Day of Year (DOY) and Ship of  Soda Can 330ml",
    labels={"doy": "Day of Year (DOY)", "total_sales": "Total Sales", "ship_id": "Ship"},
)

fig.update_layout(
    legend_title_text="Ship",
    xaxis=dict(tickmode="linear"),
)

fig.show()


## Seasonal decompositions by ship, for  Soda Can 330ml with periods 7 and 30

In [4]:
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# Prepare data
_df = df.sort_values(["doy", "ship_id"]).copy()
ship_ids = list(pd.unique(_df["ship_id"]))
periods = [7, 30]

# Precompute decompositions for each ship and period
decomposed = {}  # key: (ship, period) -> dict with components
for ship in ship_ids:
    sdata = _df[_df["ship_id"] == ship].sort_values("doy")
    y = sdata["total_sales"].astype(float)
    x = sdata["doy"].values
    for p in periods:
        try:
            if len(y) < 2 * p:
                # Not enough data points for seasonal_decompose; skip
                continue
            result = seasonal_decompose(y, model="additive", period=p, extrapolate_trend="freq")
            decomposed[(ship, p)] = {
                "x": x,
                "observed": y.values,
                "trend": result.trend,
                "seasonal": result.seasonal,
                "resid": result.resid,
            }
        except Exception:
            # Skip combinations that fail decomposition
            continue

# Build figure with one set of subplots and toggle traces via dropdown
fig = make_subplots(
    rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05,
    subplot_titles=("Observed", "Trend", "Seasonal", "Residual")
)

trace_groups = []  # list of [(obs_idx, trend_idx, seas_idx, resid_idx)] per (ship, period)
labels = []

for (ship, p), comp in decomposed.items():
    x = comp["x"]
    # Observed
    obs_trace = go.Scatter(x=x, y=comp["observed"], mode="lines+markers", name=f"{ship} Observed (P={p})",
                           hovertemplate="DOY=%{x}<br>Observed=%{y}<extra></extra>")
    fig.add_trace(obs_trace, row=1, col=1)
    obs_idx = len(fig.data) - 1

    # Trend
    trend_trace = go.Scatter(x=x, y=comp["trend"], mode="lines", name=f"{ship} Trend (P={p})",
                             hovertemplate="DOY=%{x}<br>Trend=%{y}<extra></extra>")
    fig.add_trace(trend_trace, row=2, col=1)
    trend_idx = len(fig.data) - 1

    # Seasonal
    seas_trace = go.Scatter(x=x, y=comp["seasonal"], mode="lines", name=f"{ship} Seasonal (P={p})",
                            hovertemplate="DOY=%{x}<br>Seasonal=%{y}<extra></extra>")
    fig.add_trace(seas_trace, row=3, col=1)
    seas_idx = len(fig.data) - 1

    # Residual
    resid_trace = go.Scatter(x=x, y=comp["resid"], mode="lines+markers", name=f"{ship} Residual (P={p})",
                             hovertemplate="DOY=%{x}<br>Residual=%{y}<extra></extra>")
    fig.add_trace(resid_trace, row=4, col=1)
    resid_idx = len(fig.data) - 1

    trace_groups.append((obs_idx, trend_idx, seas_idx, resid_idx))
    labels.append(f"{ship} • P={p}")

# If nothing to show, create an empty figure
if not trace_groups:
    fig = go.Figure()
    fig.update_layout(title="No seasonal decomposition available (insufficient data)")
    fig.show()
else:
    # Set all traces invisible except the first group
    visible = [False] * len(fig.data)
    for idx in trace_groups[0]:
        visible[idx] = True

    fig.update_traces(visible=False)
    for idx in trace_groups[0]:
        fig.data[idx].visible = True

    # Dropdown to select (ship, period) combination
    buttons = []
    for i, label in enumerate(labels):
        vis = [False] * len(fig.data)
        for idx in trace_groups[i]:
            vis[idx] = True
        # Extract ship and period for title
        ship_label, p_part = label.split(" • ")
        p_val = int(p_part.split("=")[1])
        buttons.append(dict(
            label=label,
            method="update",
            args=[
                {"visible": vis},
                {"title": f"Seasonal Decomposition for {ship_label} (Period={p_val})"}
            ],
        ))

    fig.update_layout(
        title=f"Seasonal Decomposition for {labels[0].split(' • ')[0]} (Period={int(labels[0].split('=')[1])})",
        height=900,
        updatemenus=[dict(
            type="dropdown",
            x=0.0, y=1.15,
            xanchor="left", yanchor="top",
            buttons=buttons,
            showactive=True
        )],
        legend_title_text="Components",
        margin=dict(t=120)
    )
    fig.update_xaxes(title_text="Day of Year (DOY)", row=4, col=1)
    fig.update_yaxes(title_text="Observed", row=1, col=1)
    fig.update_yaxes(title_text="Trend", row=2, col=1)
    fig.update_yaxes(title_text="Seasonal", row=3, col=1)
    fig.update_yaxes(title_text="Residual", row=4, col=1)

    fig.show()


## 14 days forecast by ship for Soda Can 330ml, using SARIMAX

In [20]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.statespace.sarimax import SARIMAX

horizon = 14
reference_year = 2025

# Build dated time series with explicit daily frequency per ship
ts_df = df.copy()
ts_df["date"] = pd.Timestamp(f"{reference_year}-01-01") + pd.to_timedelta(ts_df["doy"] - 1, unit="D")
ts_df = ts_df.sort_values(["ship_id", "date"])

forecast_frames = []

for ship in ts_df["ship_id"].unique():
    sdata = ts_df[ts_df["ship_id"] == ship][["date", "total_sales"]].set_index("date").sort_index()

    if sdata.empty:
        continue

    # Ensure continuous daily index
    full_idx = pd.date_range(start=sdata.index.min(), end=sdata.index.max(), freq="D")
    y = sdata["total_sales"].reindex(full_idx, fill_value=0.0)
    try:
        y.index.freq = y.index.inferred_freq or "D"
    except Exception:
        pass

    # Exogenous regressors: day-of-week dummies
    dow_onehot = pd.get_dummies(y.index.dayofweek, prefix="dow")
    exog_fit = dow_onehot

    made_fc = False
    try:
        # Use fixed seasonal SARIMA structure
        order = (1, 1, 1) if len(y) > 8 else (0, 1, 1)
        seasonal_order = (1, 0, 1, 7)  # weekly seasonality

        model = SARIMAX(
            y,
            exog=exog_fit,
            order=order,
            seasonal_order=seasonal_order,
            trend="c",
            enforce_stationarity=False,
            enforce_invertibility=False,
        )
        res = model.fit(disp=False)

        last_date = y.index.max()
        future_idx = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=horizon, freq="D")
        future_dow = pd.get_dummies(future_idx.dayofweek, prefix="dow")

        # Forecast with exogenous inputs
        fc_res = res.get_forecast(steps=horizon, exog=future_dow)
        fc_mean = fc_res.predicted_mean
        fc_ci = fc_res.conf_int(alpha=0.2)  # 80% CI

        fc_mean.index = future_idx
        fc_ci.index = future_idx

        fc_frame = pd.DataFrame(
            {
                "date": future_idx,
                "ship_id": ship,
                "forecast": fc_mean.values,
                "lower80": fc_ci.iloc[:, 0].values,
                "upper80": fc_ci.iloc[:, 1].values,
            }
        )
        fc_frame["doy"] = fc_frame["date"].dt.dayofyear
        forecast_frames.append(fc_frame)
        made_fc = True
    except Exception:
        made_fc = False

    # Fallback 1: seasonal naive (weekly)
    if not made_fc and len(y) >= 7:
        last_7 = y.iloc[-7:].values
        reps = (horizon + 6) // 7
        fc_vals = (list(last_7) * reps)[:horizon]
        future_idx = pd.date_range(start=y.index.max() + pd.Timedelta(days=1), periods=horizon, freq="D")
        fc_frame = pd.DataFrame(
            {"date": future_idx, "ship_id": ship, "forecast": fc_vals, "lower80": np.nan, "upper80": np.nan}
        )
        fc_frame["doy"] = fc_frame["date"].dt.dayofyear
        forecast_frames.append(fc_frame)

    # Fallback 2: repeat last value
    elif not made_fc and len(y) >= 1:
        last_val = float(y.iloc[-1])
        future_idx = pd.date_range(start=y.index.max() + pd.Timedelta(days=1), periods=horizon, freq="D")
        fc_frame = pd.DataFrame(
            {"date": future_idx, "ship_id": ship, "forecast": [last_val] * horizon, "lower80": np.nan, "upper80": np.nan}
        )
        fc_frame["doy"] = fc_frame["date"].dt.dayofyear
        forecast_frames.append(fc_frame)

forecast_sarimax_df = (
    pd.concat(forecast_frames, ignore_index=True)
    if forecast_frames
    else pd.DataFrame(columns=["date", "ship_id", "forecast", "lower80", "upper80", "doy"])
)

# ------------------- PLOT SECTION -------------------
if not forecast_sarimax_df.empty:
    # Prepare data for line plot
    _hist_plot = ts_df.rename(columns={"total_sales": "value"})[["date", "ship_id", "value"]].copy()
    _hist_plot["kind"] = "Actual"

    _fc_plot = forecast_sarimax_df[["date", "ship_id", "forecast"]].rename(columns={"forecast": "value"}).copy()
    _fc_plot["kind"] = "Forecast"

    _plot_df = pd.concat([_hist_plot, _fc_plot], ignore_index=True)

    # Base line plot with Plotly Express
    fig_sarimax = px.line(
        _plot_df,
        x="date",
        y="value",
        color="ship_id",
        line_dash="kind",
        title="14-Day Forecast per Ship (Manual SARIMAX + DOW exogenous)",
        labels={"date": "Date", "value": "Total Sales", "ship_id": "Ship"},
    )

    # Get colors used by Plotly Express for each ship
    ship_colors = {trace.name.split(",")[0]: trace.line.color for trace in fig_sarimax.data}

    # Add confidence intervals as shaded regions
    for ship in forecast_sarimax_df["ship_id"].unique():
        ship_df = forecast_sarimax_df[forecast_sarimax_df["ship_id"] == ship].dropna(subset=["lower80", "upper80"])
        if not ship_df.empty:
            color = ship_colors.get(ship, "gray")
            fig_sarimax.add_trace(
                go.Scatter(
                    x=pd.concat([ship_df["date"], ship_df["date"][::-1]]),
                    y=pd.concat([ship_df["lower80"], ship_df["upper80"][::-1]]),
                    fill="toself",
                    fillcolor=color.replace("rgb", "rgba").replace(")", ",0.2)"),  # transparent version of line color
                    line=dict(color="rgba(255,255,255,0)"),
                    hoverinfo="skip",
                    name=f"{ship} 80% CI",
                    showlegend=True,
                )
            )

    # Ensure forecast/actual lines are on top of shaded regions
    fig_sarimax.update_traces(mode="lines+markers")
    fig_sarimax.update_layout(legend_title_text="Ship / Series")
    fig_sarimax.show()
else:
    print("No forecasts available.")
